# Advanced Customization and Features

This notebook covers advanced features like custom paths, different queue models, and extending the simulation.

## Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import json
from cascabel.models.waitline import WaitLine
from cascabel.models.simulation import Simulation
from cascabel.models.models import BorderCrossingConfig, SimulationConfig

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

## Working with Different GeoJSON Paths

The simulation uses GeoJSON files to define border crossing paths. Let's explore the available paths.

In [ ]:
# List available paths
import os

paths_dir = "cascabel/paths"
available_paths = []

for root, dirs, files in os.walk(paths_dir):
    for file in files:
        if file.endswith('.geojson'):
            rel_path = os.path.relpath(os.path.join(root, file), paths_dir)
            available_paths.append(rel_path)

print("Available GeoJSON paths:")
for path in available_paths:
    print(f"- {path}")

In [ ]:
# Load and visualize different paths
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

paths_to_compare = [
    "usa2mx/bota.geojson",
    "mx2usa/bota.geojson"
]

colors = ['blue', 'red']
labels = ['USA to Mexico', 'Mexico to USA']

for i, (path_name, color, label) in enumerate(zip(paths_to_compare, colors, labels)):
    try:
        full_path = os.path.join(paths_dir, path_name)
        gdf = gpd.read_file(full_path)
        
        # Plot the geometry
        for geom in gdf.geometry:
            if isinstance(geom, LineString):
                x, y = geom.xy
                axes[i].plot(x, y, color=color, linewidth=3, label=label)
            elif hasattr(geom, 'geoms'):  # MultiLineString
                for line in geom.geoms:
                    x, y = line.xy
                    axes[i].plot(x, y, color=color, linewidth=3, label=label)
                    break  # Only plot first segment for clarity
        
        axes[i].set_title(f'{label} Path')
        axes[i].set_xlabel('Longitude')
        axes[i].set_ylabel('Latitude')
        axes[i].grid(True)
        axes[i].legend()
        
    except Exception as e:
        print(f"Error loading {path_name}: {e}")

plt.tight_layout()
plt.show()

## Creating Custom Paths

You can create custom GeoJSON paths for different border crossings.

In [ ]:
# Create a simple custom path
custom_coordinates = [
    [-106.5, 31.8],  # Start point
    [-106.4, 31.7],
    [-106.3, 31.6],
    [-106.2, 31.5],
    [-106.1, 31.4]   # End point
]

# Create GeoJSON structure
custom_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Custom Border Path"
            },
            "geometry": {
                "type": "LineString",
                "coordinates": custom_coordinates
            }
        }
    ]
}

# Save to file
custom_path_file = "cascabel/paths/custom_path.geojson"
with open(custom_path_file, 'w') as f:
    json.dump(custom_geojson, f, indent=2)

print(f"Custom path saved to {custom_path_file}")

# Visualize
fig, ax = plt.subplots()
lons, lats = zip(*custom_coordinates)
ax.plot(lons, lats, 'g-', linewidth=3, marker='o')
ax.scatter(lons[0], lats[0], c='green', s=100, label='Start')
ax.scatter(lons[-1], lats[-1], c='red', s=100, label='End')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Custom Border Crossing Path')
ax.legend()
ax.grid(True)
plt.show()

## Using Custom Paths in Simulation

In [ ]:
# Test simulation with custom path
waitline = WaitLine(
    geojson_path=custom_path_file,
    speed_regime={"slow": 0.8, "fast": 0.2},
    line_length_seed=0.5
)

print(f"Custom path loaded: {len(waitline.path)} points, {waitline.total_length:.2f}m total length")

# Run a quick simulation
border_config = BorderCrossingConfig(
    num_queues=1,
    nodes_per_queue=[1],
    arrival_rate=0.5,
    service_rates=[0.8],
    queue_assignment='shortest',
    safe_distance=8.0,
    max_queue_length=50
)

simulation_config = SimulationConfig(
    max_simulation_time=600.0,  # 10 minutes
    time_factor=1.0,
    enable_telemetry=True,
    enable_position_tracking=True
)

simulation = Simulation(
    waitline=waitline,
    border_config=border_config,
    simulation_config=simulation_config
)

simulation()
stats = simulation.get_statistics()

print("\nSimulation Results with Custom Path:")
print(f"Total arrivals: {stats.execution_stats.total_arrivals}")
print(f"Average wait time: {stats.execution_stats.average_wait_time:.2f}s")
print(f"Throughput: {stats.execution_stats.total_completions / stats.simulation_duration * 3600:.1f} cars/hour")

## Exploring Queue Assignment Strategies

Different strategies for assigning cars to queues.

In [ ]:
# Compare queue assignment strategies
strategies = ['shortest', 'random', 'round_robin']
results = []

for strategy in strategies:
    print(f"Testing {strategy} assignment...")
    
    border_config = BorderCrossingConfig(
        num_queues=3,
        nodes_per_queue=[1, 1, 1],
        arrival_rate=1.5,
        service_rates=[0.8, 0.9, 0.7],
        queue_assignment=strategy,
        safe_distance=8.0,
        max_queue_length=50
    )
    
    simulation = Simulation(
        waitline=WaitLine("cascabel/paths/usa2mx/bota.geojson", {"slow": 0.8, "fast": 0.2}, 0.5),
        border_config=border_config,
        simulation_config=SimulationConfig(
            max_simulation_time=1800.0,
            time_factor=1.0,
            enable_telemetry=False,
            enable_position_tracking=False
        )
    )
    
    simulation()
    stats = simulation.get_statistics()
    
    results.append({
        'strategy': strategy,
        'avg_wait_time': stats.execution_stats.average_wait_time,
        'avg_queue_length': stats.execution_stats.average_queue_length,
        'utilization': stats.execution_stats.overall_utilization
    })

df_strategies = pd.DataFrame(results)
df_strategies

In [ ]:
# Visualize strategy comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

x = np.arange(len(df_strategies))
width = 0.25

axes[0].bar(x, df_strategies['avg_wait_time'], width, label='Wait Time')
axes[0].set_ylabel('Average Wait Time (s)')
axes[0].set_title('Wait Time by Assignment Strategy')
axes[0].set_xticks(x)
axes[0].set_xticklabels(df_strategies['strategy'])

axes[1].bar(x, df_strategies['avg_queue_length'], width, color='orange', label='Queue Length')
axes[1].set_ylabel('Average Queue Length')
axes[1].set_title('Queue Length by Assignment Strategy')
axes[1].set_xticks(x)
axes[1].set_xticklabels(df_strategies['strategy'])

axes[2].bar(x, df_strategies['utilization'], width, color='green', label='Utilization')
axes[2].set_ylabel('Utilization')
axes[2].set_title('Utilization by Assignment Strategy')
axes[2].set_xticks(x)
axes[2].set_xticklabels(df_strategies['strategy'])

plt.tight_layout()
plt.show()

## Extending the Simulation

Ideas for extending Cascabel:

1. **Time-varying arrival rates**: Model rush hours vs off-peak times
2. **Weather effects**: Reduce service rates during bad weather
3. **Vehicle types**: Different processing times for cars vs trucks
4. **Dynamic lane opening**: Open/close lanes based on demand
5. **Real-time adaptation**: Adjust strategies based on current conditions

## Example: Time-varying Arrival Rates

In [ ]:
# Simulate time-varying arrival rates (rush hour effect)
import numpy as np

# Create arrival rate that varies over time (sine wave)
time_points = np.linspace(0, 24, 100)  # 24 hours
base_rate = 1.0
amplitude = 0.8
arrival_rates = base_rate + amplitude * np.sin(2 * np.pi * (time_points - 6) / 24)  # Peak at 6 AM
arrival_rates = np.maximum(arrival_rates, 0.1)  # Minimum rate

# Plot the arrival rate pattern
fig, ax = plt.subplots()
ax.plot(time_points, arrival_rates, 'b-', linewidth=2)
ax.fill_between(time_points, arrival_rates, alpha=0.3)
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Arrival Rate (cars/hour)')
ax.set_title('Time-Varying Arrival Rates (Rush Hour Pattern)')
ax.grid(True)
ax.axvline(6, color='red', linestyle='--', alpha=0.7, label='Peak Time')
ax.legend()
plt.show()

print(f"Peak arrival rate: {arrival_rates.max():.2f} cars/hour")
print(f"Minimum arrival rate: {arrival_rates.min():.2f} cars/hour")
print(f"Average arrival rate: {arrival_rates.mean():.2f} cars/hour")

## Next Steps

Now that you've explored the advanced features:

- Try creating your own GeoJSON paths
- Experiment with different queue configurations
- Implement time-varying parameters
- Consider contributing new features to the project!

The Cascabel codebase is designed to be extensible, so feel free to modify and enhance it for your specific use cases.